In [3]:
# default_exp mrna_content

# mRNA content regression

> API details.

In [4]:
#hide
from nbdev.showdoc import *

In [5]:
# %load common.py
# export

import pandas
import os
import numpy
import seaborn
import logging
import matplotlib.pyplot as plt
from importlib import reload
from joblib import load, dump
from sklearn.model_selection import PredefinedSplit
from sklearn import metrics

numpy.random.seed(42)

In [6]:
# export

import time
import fcsparser
import sklearn.linear_model
import sklearn.model_selection
from calibration_singlecell import classification
from functools import partial
from ehv import mrna_content_regression
import scipy.stats

In [7]:
meta, df = fcsparser.parse("D:/Experiment_data/weizmann/EhV/201/aggregates/non_intensity_T0234567_R123.fcs")
df_meta = df[["label", "replicate", "timepoint"]].astype(int)
df_meta["timepoint"] = df_meta["timepoint"].map(lambda a: meta["timepoint"].split(",")[a])
df_meta["replicate"] = df_meta["replicate"].map(lambda a: meta["replicate"].split(",")[a])
df = df.drop(columns=["label", "replicate", "timepoint"])

In [11]:
_, fluor_df = fcsparser.parse("D:/Experiment_data/weizmann/EhV/201/aggregates/fluor_T0234567_R123.fcs")

In [8]:
metric_funcs = [
    metrics.explained_variance_score,
    metrics.max_error,
    metrics.mean_absolute_error,
    metrics.mean_squared_error,
    metrics.mean_squared_error,
    metrics.median_absolute_error,
    metrics.r2_score
]

# BF regression on targets

In [12]:
bf_df = df.filter(regex="(i?).*BF.*")

# Prepare folds

In [13]:
Xy_train = []
Xy_nested_train = []
Xy_test = []

target_col = "IntensityMCTMRlogicle"

test_fold, nested_test_folds = load("D:/Experiment_data/weizmann/EhV/201/5x3_samplesplit.dat")

for i, (train_idx, test_idx) in enumerate(PredefinedSplit(test_fold).split()):
    
    X = bf_df.iloc[train_idx]
    mean, std = X.mean(), X.std()
    
    y_mean = fluor_df[target_col].iloc[train_idx].mean()
    
    Xy_train.append(((X - mean) / std, fluor_df[target_col].iloc[train_idx] - y_mean))
    Xy_test.append(((bf_df.iloc[test_idx] - mean) / std, fluor_df[target_col].iloc[test_idx] - y_mean))
    
    Xy_train_tmp = []
    for nested_train_idx, nested_val_idx in PredefinedSplit(nested_test_folds[i]).split():
        
        X_train = bf_df.iloc[train_idx[nested_train_idx]]
        mean, std = X.mean(), X.std()
        
        X_val = bf_df.iloc[train_idx[nested_val_idx]]
        
        y_mean = fluor_df[target_col].iloc[train_idx[nested_train_idx]].mean()
        
        Xy_train_tmp.append((
            ((X_train - mean) / std, fluor_df[target_col].iloc[train_idx[nested_train_idx]] - y_mean),
            ((X_val - mean) / std, fluor_df[target_col].iloc[train_idx[nested_val_idx]] - y_mean)
        ))
        
    Xy_nested_train.append(Xy_train_tmp)